In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
from datetime import datetime

print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
from DateProvider import DateProvider
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
def getWorkingDirectory():
    pwd = ! "pwd"
    return pwd[0]

In [ ]:
pwd = getWorkingDirectory()

In [ ]:
pwd

## Download VAERS-Data

In [ ]:
# FK-TODO: das Modell erst dann laden, wenn es wirklich gebraucht wird
from VAERSFileDownloader import downloadVAERSFileAndUnzip

In [ ]:
if needsUpdate:
    workingDirectory = getWorkingDirectory()
    downloadVAERSFileAndUnzip('2022VAERSData.zip', workingDirectory)
    downloadVAERSFileAndUnzip('NonDomesticVAERSData.zip', workingDirectory)

In [ ]:
from BatchCodeTableHtmlUpdater import updateBatchCodeTableHtmlFile

In [ ]:
from InternationalVaersCovid19Provider import getInternationalVaersCovid19

In [ ]:
internationalVaersCovid19 = getInternationalVaersCovid19(years = [2020, 2021, 2022])
internationalVaersCovid19

In [ ]:
updateBatchCodeTableHtmlFile(internationalVaersCovid19)

In [ ]:
def publishGitHubPages():
    %cd /home/frankknoll/Dokumente/Corona/projects/HowBadIsMyBatch-pages
    ! git add -A
    ! git commit -m "updating data for batch codes"
    ! git push

### see https://knollfrank.github.io/HowBadIsMyBatch/batchCodeTable.html

In [ ]:
publishGitHubPages()